In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
!curl -O http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.9M  100 46.9M    0     0  6890k      0  0:00:06  0:00:06 --:--:-- 9972k


# 第7章: データベース
artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．
(グラフ省略)
artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [0]:
# https://qiita.com/KurosawaTsuyoshi/items/f8719bf7c3a10d22a921#%E3%82%BD%E3%83%BC%E3%82%B9%E3%81%8B%E3%82%89%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB%E3%81%99%E3%82%8B%E5%A0%B4%E5%90%88
# Redis install
!wget http://download.redis.io/releases/redis-3.0.7.tar.gz
!tar xzf redis-3.0.7.tar.gz
!cd redis-3.0.7 && make && make install
# add Redis user
!groupadd redis
!useradd -s /sbin/nologin -M -g redis redis
# create setting dir
!mkdir /etc/redis /var/run/redis /var/log/redis
!chmod 755 /etc/redis /var/run/redis /var/log/redis
!chown redis:redis /etc/redis /var/run/redis /var/log/redis

In [0]:
# setting file
!cp -p redis_init_script /etc/init.d/redis
!chmod 700 /etc/init.d/redis
!cp -p 6379.conf /etc/redis/6379.conf
!chmod 700 /etc/redis/6379.conf

In [3]:
# Redis run
!/etc/init.d/redis start

Starting Redis server...


In [6]:
# check runing Redis.d
!ps -e | grep redis

   3125 ?        00:00:00 redis-server


In [0]:
!pip install redis
# https://redis-py.readthedocs.io/en/latest/

In [31]:

import redis

import gzip
import json


class KVS(object):

    def __init__(self):
        self.db = None

class RedisOperator(KVS):

    def __init__(self):
        self.init_redis()

    def init_redis(self):
        self.db = redis.StrictRedis(password="hoge")

    def set_data(self, key, value):
        if type(key) == list:
            self.db.rpush(key, value)
        else:
            self.db.set(key, value)

    def load_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "area" in obj:
                    self.set_data(obj["name"], obj["area"])
                else:
                    self.set_data(obj["name"], "")
                data = f.readline()
        print('load done ...!')


gz_json = "artist.json.gz"
r = RedisOperator()
r.load_data_from_gzip_json_file(gz_json)

load done ...!


## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ

In [33]:

import redis

import gzip
import json


class KVS(object):

    def __init__(self):
        self.db = None

class RedisOperator(KVS):

    def __init__(self):
        self.init_redis()

    def init_redis(self):
        self.db = redis.StrictRedis(password="hoge")

    def set_data(self, key, value):
        if type(key) == list:
            self.db.rpush(key, value)
        else:
            self.db.set(key, value)

    def load_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "area" in obj:
                    self.set_data(obj["name"], obj["area"])
                else:
                    self.set_data(obj["name"], "")
                data = f.readline()
        print('load done ...!')

    def fetch_area_data(self, search_value):
        result = self.db.get(search_value)
        if not result:
            return None
        return result.decode()


gz_json = "artist.json.gz"
r = RedisOperator()
search_value = input('活動地域を取得したいアーティスト名を入力してください: ')
result = r.fetch_area_data(search_value)
print(result)

活動地域を取得したいアーティスト名を入力してください: Shabba Ranks
Jamaica


## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [38]:

import redis

import gzip
import json


class KVS(object):

    def __init__(self):
        self.db = None

class RedisOperator(KVS):

    def __init__(self):
        self.init_redis()

    def init_redis(self):
        self.db = redis.StrictRedis(password="hoge")

    def set_data(self, key, value):
        if type(key) == list:
            self.db.rpush(key, value)
        else:
            self.db.set(key, value)

    def load_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "area" in obj:
                    self.set_data(obj["name"], obj["area"])
                else:
                    self.set_data(obj["name"], "")
                data = f.readline()
        print('load done ...!')

    def fetch_area_data(self, search_value):
        result = self.db.get(search_value)
        if not result:
            return None
        return result.decode()

    def count_area_data(self, area_value):
        return self.db.strlen(area_value)


gz_json = "artist.json.gz"
r = RedisOperator()
search_value = input('取得したい件数の活動地域名を入力してください: ')
result = r.count_area_data(search_value)
print(result)

取得したい件数の活動地域名を入力してください: Japan
14


## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [50]:

import redis

import gzip
import json


class KVS(object):

    def __init__(self):
        self.db = None

class RedisOperator(KVS):

    def __init__(self):
        self.init_redis()

    def init_redis(self):
        self.db = redis.StrictRedis(password="hoge", db=2)

    def set_data(self, key, value):
        if type(key) == list:
            self.db.rpush(key, value)
        else:
            self.db.set(key, value)

    def set_hash_data(self, p_key, c_key, c_value):
        self.db.hset(p_key, c_key, c_value)

    def load_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "area" in obj:
                    self.set_data(obj["name"], obj["area"])
                else:
                    self.set_data(obj["name"], "")
                data = f.readline()
        print('load done ...!')

    def load_hash_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "tags" in obj:
                    self.set_hash_data(obj["name"], obj["id"], json.dumps(obj["tags"]))
                else:
                    self.set_hash_data(obj["name"], obj["id"], "")
                data = f.readline()
        print('load done ...!')


    def fetch_area_data(self, search_value):
        result = self.db.get(search_value)
        if not result:
            return None
        return result.decode()

    def count_area_data(self, area_value):
        return self.db.strlen(area_value)


gz_json = "artist.json.gz"
r = RedisOperator()
#search_value = input('取得したい件数の活動地域名を入力してください: ')
r.load_hash_data_from_gzip_json_file(gz_json)


load done ...!


In [57]:

import redis

import gzip
import json


class KVS(object):

    def __init__(self):
        self.db = None

class RedisOperator(KVS):

    def __init__(self):
        self.init_redis()

    def init_redis(self):
        self.db = redis.StrictRedis(password="hoge", db=2)

    def set_data(self, key, value):
        if type(key) == list:
            self.db.rpush(key, value)
        else:
            self.db.set(key, value)

    def set_hash_data(self, p_key, c_key, c_value):
        self.db.hset(p_key, c_key, c_value)

    def load_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "area" in obj:
                    self.set_data(obj["name"], obj["area"])
                else:
                    self.set_data(obj["name"], "")
                data = f.readline()
        print('load done ...!')

    def load_hash_data_from_gzip_json_file(self, json_file_path):
        with gzip.open(json_file_path, 'r') as f:
            data = f.readline()
            while data:
                obj = json.loads(data)
                if "tags" in obj:
                    self.set_hash_data(obj["name"], obj["id"], json.dumps(obj["tags"]))
                else:
                    self.set_hash_data(obj["name"], obj["id"], "")
                data = f.readline()
        print('load done ...!')


    def fetch_area_data(self, search_value):
        result = self.db.get(search_value)
        if not result:
            return None
        return result.decode()

    def count_area_data(self, area_value):
        return self.db.strlen(area_value)

    def fetch_hash_data(self, search_value):
        return self.db.hgetall(search_value)

gz_json = "artist.json.gz"
r = RedisOperator()
search_value = input('取得したいアーティスト名を入力してください: ')
print(r.fetch_hash_data(search_value))


取得したいアーティスト名を入力してください: Shabba Ranks
{b'37082': b'[{"count": 1, "value": "raggamuffin"}, {"count": 1, "value": "dancehall"}, {"count": 1, "value": "pop"}, {"count": 1, "value": "pop and chart"}, {"count": 1, "value": "drum and bass"}, {"count": 1, "value": "jungle"}, {"count": 1, "value": "mc"}]'}


## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [76]:
!apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 9DA31620334BD75D9DCB49F368818C72E52529D4
#!echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-4.2.list
!echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.0.list
!apt-get update && apt-get install -y mongodb-org

Executing: /tmp/apt-key-gpghome.hdOuN9E2fn/gpg.1.sh --keyserver hkp://keyserver.ubuntu.com:80 --recv 9DA31620334BD75D9DCB49F368818C72E52529D4
gpg: key 68818C72E52529D4: public key "MongoDB 4.0 Release Signing Key <packaging@mongodb.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1
deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.0 multiverse
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:6 https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.0 InRelease
Ign:7 https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:9 https://repo.mongodb.org/apt/ubuntu bionic/mo

In [0]:
!ps -e | grep mong

In [0]:
!mongo

In [0]:
!

In [77]:
!systemctl enable mongod
!systemctl mongod start

Unknown operation mongod.
